In [1]:
# Set up the notebook to import modules from relative paths
import os, sys

#'/home/user/example/parent/child'
current_path = os.path.abspath('.')

#'/home/user/example/parent'
parent_path = os.path.dirname(current_path)

sys.path.append(parent_path)

In [2]:
from data_pipeline import ETL_Pipeline 

dp = ETL_Pipeline('/workspace/shared-data/email-campaign/')
transformed_df = dp.process('sent_emails.csv','responded.csv','userbase.csv')
transformed_df.head()

,SubjectLine_ID,Gender,Type,Email_Domain,Tenure_Group,Age_Group,Sent_Day,Response_Received,Sent_Emails
0,1,0,0,0,0,1,1,0,1
1,1,0,0,0,0,1,3,3,3
2,1,0,0,0,0,1,6,2,3
3,1,0,0,0,0,2,0,0,1
4,1,0,0,0,0,2,2,0,1


In [3]:
import pandas as pd
import numpy as np
import sklearn

from IPython.display import display, HTML

# Display Properties
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
df = pd.read_csv('/workspace/shared-data/email-campaign/email_campaign_data.csv')
df.head()

,SubjectLine_ID,Gender,Type,Email_Domain,Tenure_Group,Age_Group,Sent_Day,Response_Received,Sent_Emails
0,1,0,0,0,0,1,1,0,1
1,1,0,0,0,0,1,3,3,3
2,1,0,0,0,0,1,6,2,3
3,1,0,0,0,0,2,0,0,1
4,1,0,0,0,0,2,2,0,1


In [5]:
from dataset import Email_Dataset

# Create a email campaign dataset with 5 folds 
ecd = Email_Dataset(df,'Response_Received')

# Get training dataset for a fold
training_df = ecd.get_training_dataset(2)
training_df.head()

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,SubjectLine_ID,Gender,Type,Email_Domain,Age_Group,Tenure_Group,Sent_Day,Sent_Emails,Response_Received
0,1,0,0,0,0,1,1,1,0
1,1,0,0,0,0,1,3,3,3
2,1,0,0,0,0,1,6,3,2
3,1,0,0,0,0,2,0,1,0
4,1,0,0,0,0,2,2,1,0


In [6]:
# Get testing dataset for a fold
testing_df = ecd.get_testing_dataset(2)
testing_df.head()

,SubjectLine_ID,Gender,Type,Email_Domain,Age_Group,Tenure_Group,Sent_Day,Sent_Emails,Response_Received
0,1,0,0,2,2,3,4,600,245
1,1,0,0,2,2,3,6,580,244
2,1,0,1,1,2,2,4,601,214
3,1,0,1,2,1,2,6,756,286
4,1,0,1,2,2,2,3,2161,853


In [7]:
print(f"We have {len(training_df)} training responses and {len(testing_df)} testing responses")

We have 11882 training responses and 2970 testing responses


In [8]:
from model import Email_Campaign_Model
from metrics import Metrics

metrics = Metrics()

# Train the Model using entire dataset and with defaults of conversion threshold of 30% and sent emails as 10
model = Email_Campaign_Model(df)

Here we will build the state table


In [9]:
qtable_df = model.train(iterations=20000, starting_state = (1,0,0,0,0,0,0), epsilon = 0.1, alpha = 0.1,gamma = 0.6)
qtable_df.head()

,Day of Week,Tenure Group,Email Domain,Age Group,Gender,Type,Subject Id
0,-0.100,5.590,0.000,0.000,0.000,0.000,0.000
1,-0.100,0.000,0.000,0.000,0.000,0.000,0.000
2,-0.100,3.111,0.000,0.000,0.000,0.000,-0.100
3,-0.100,5.967,0.000,0.000,0.000,0.000,0.000
4,-0.100,-0.100,-0.100,-0.100,-0.100,0.000,0.000


In [10]:
qtable_df.to_csv('q_table.csv')

In [11]:
# Load the model again but now with the Q Table generated from training and we now don't need to consider the email sent 
# threshold as we have trained the model to get Q Table Values using that
saved_model = Email_Campaign_Model(df, (1,0,0,0,0,0,0), 1, 0.25, 'q_table.csv', model.get_states())

# Run metrics using this Q Table across different splits
folds = []
avg_conv = []
median_conv = []
min_conv = []
max_conv = [] 

# Run through all the folds
for i in range(5):
    # Get training for the fold and get metrics
    train = ecd.get_training_dataset(i)
    avg, median, min_conversion, max_conversion = saved_model.test(train,10)
    folds += ['Training Fold ' + str(i)]
    avg_conv += [avg]
    median_conv += [median]
    min_conv += [min_conversion]
    max_conv += [max_conversion] 

metrics.generate_report(folds,avg_conv,median_conv,min_conv,max_conv,'../results/training-results.txt')

In [14]:
# Run metrics using this Q Table across different splits
folds = []
avg_conv = []
median_conv = []
min_conv = []
max_conv = [] 

# Run through all the folds
for i in range(5):
    # Get training for the fold and get metrics
    test = ecd.get_testing_dataset(i)
    avg, median, min_conversion, max_conversion = saved_model.test(test,10)
    folds += ['Training Fold ' + str(i)]
    avg_conv += [avg]
    median_conv += [median]
    min_conv += [min_conversion]
    max_conv += [max_conversion] 

metrics.generate_report(folds,avg_conv,median_conv,min_conv,max_conv,'../results/testing-results.txt')